<a href="https://colab.research.google.com/github/subhendu115/Spark/blob/main/multiplt-delimeter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=f5f1001561d2dc4930a78a025a269310e69f35c978c3ba413669a4e765e4f7f3
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [5]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [6]:
from pyspark.sql import SparkSession

In [13]:
spark = SparkSession.builder.appName("MultiDelim").master("local[*]").getOrCreate()
import os;print(os.getcwd())

/content


In [23]:
basePath = "/content/gdrive/MyDrive/SparkData/dataset-master/"

In [24]:
# Multiple delimeter file
df = spark.read.format("csv").option("sep","~|").option("header",True).load(basePath+"/multi-delim.dat")

In [25]:
df.show()

+---+-----------+---+
| id|       name|Age|
+---+-----------+---+
|  1|       gsfg| 23|
|  1|  ssdgdsgjs| 20|
|  1|svfgbfbggjs| 21|
|  1|     svfscv| 29|
|  1|   svcvvbjs| 23|
|  1| ssgsrhjbjs| 28|
|  1|  svfhfhbjs| 23|
|  1|  svfdgdbjs| 25|
|  1|   svfdgbjs| 22|
|  1| svdgngjbjs| 25|
+---+-----------+---+



In [26]:
df1 = spark.read.format("text").option("header",True).load(basePath+"/multi-delim.dat")

In [ ]:
df1.show()

+------------------+
|             value|
+------------------+
|     id~|name~|Age|
|       1~|gsfg~|23|
|  1~|ssdgdsgjs~|20|
|1~|svfgbfbggjs~|21|
|     1~|svfscv~|29|
|   1~|svcvvbjs~|23|
| 1~|ssgsrhjbjs~|28|
|  1~|svfhfhbjs~|23|
|  1~|svfdgdbjs~|25|
|   1~|svfdgbjs~|22|
| 1~|svdgngjbjs~|25|
+------------------+



In [20]:
header = df1.first()[0]
header
schm = header.split('~|')
schm

['id', 'name', 'Age']

In [21]:
df_in = df1.filter(df1['value'] != header).rdd.map(lambda row:row[0].split("~|")).toDF(schm)
df_in.show()
# As the DataFarme have only One column -> hence row[0]. It can change based on number of column in dataframe/RDD

+---+-----------+---+
| id|       name|Age|
+---+-----------+---+
|  1|       gsfg| 23|
|  1|  ssdgdsgjs| 20|
|  1|svfgbfbggjs| 21|
|  1|     svfscv| 29|
|  1|   svcvvbjs| 23|
|  1| ssgsrhjbjs| 28|
|  1|  svfhfhbjs| 23|
|  1|  svfdgdbjs| 25|
|  1|   svfdgbjs| 22|
|  1| svdgngjbjs| 25|
+---+-----------+---+



In [28]:
#Different Schema data merge
df1 = spark.read.format('csv').option('header',True).load(basePath+"input.csv")
df2 = spark.read.format('csv').option('header',True).load(basePath+"input1.csv")

In [29]:
df1.show()
df2.show()

+---------+---+
|     Name|Age|
+---------+---+
|bsfbssdds| 28|
|  bsfbssd| 25|
|  bsfbssd| 26|
|   bsfbss| 20|
|  bsfbsdf| 25|
|   bsfbss| 26|
|   bsfbsd| 23|
+---------+---+

+---------+---+------+
|     Name|Age|Gender|
+---------+---+------+
|bsfbssdds| 28|     M|
|  bsfbssd| 25|     F|
|  bsfbssd| 26|     M|
|   bsfbss| 20|     M|
|  bsfbsdf| 25|     M|
|   bsfbss| 26|     F|
|   bsfbsd| 23|     M|
+---------+---+------+



In [32]:
list1 = list(set(df1.columns) - set(df2.columns))
list2 = list(set(df2.columns) - set(df1.columns))
print(list1)
print(list2)

[]
['Gender']


In [35]:
from pyspark.sql.functions import lit
for col in list1:
  df2 = df2.withColumn(col,lit(None))
for col in list2:
  df1 = df1.withColumn(col,lit(None))

In [36]:
df = df1.unionAll(df2)
df.show()

+---------+---+------+
|     Name|Age|Gender|
+---------+---+------+
|bsfbssdds| 28|  NULL|
|  bsfbssd| 25|  NULL|
|  bsfbssd| 26|  NULL|
|   bsfbss| 20|  NULL|
|  bsfbsdf| 25|  NULL|
|   bsfbss| 26|  NULL|
|   bsfbsd| 23|  NULL|
|bsfbssdds| 28|     M|
|  bsfbssd| 25|     F|
|  bsfbssd| 26|     M|
|   bsfbss| 20|     M|
|  bsfbsdf| 25|     M|
|   bsfbss| 26|     F|
|   bsfbsd| 23|     M|
+---------+---+------+

